# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [3]:
# Import initial libraries
import numpy as np
import pandas as pd
import requests
from citipy import citipy
import time
from datetime import datetime
from config import op_weath_key

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [4]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats, lngs)

In [5]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [11]:
# Create an empty list for holding the cities
cities = []

# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    # Use the citipy module to identify the nearest city to the current coordinate
    city_obj = citipy.nearest_city(coordinate[0], coordinate[1])
    city = city_obj.city_name + "," + city_obj.country_code
    #city will be of the form "city,country_code" and can be passed to the openweather api like that
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

763

In [23]:
#print first 5 cities, last 6 cities
cities[0:5], cities[-6:len(cities)]

(['yulara,au',
  'hasaki,jp',
  'butaritari,ki',
  'punta arenas,cl',
  'busselton,au'],
 ['lethem,gy',
  'aykhal,ru',
  'mackay,au',
  'cleveland,us',
  'hoshcha,ua',
  'bagn,no'])

## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

In [27]:
# Assemble the OpenWeatherMap's API call
url = "https://api.openweathermap.org/data/2.5/weather?" + f"appid={op_weath_key}" + "&units=imperial"

## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [34]:
# test getting a city
call_url = url + "&q=" + cities[0].replace(" ","+")
city1_request = requests.get(call_url)
city1json = city1_request.json()
print(city1json)
print("lat: ", city1json["coord"]["lat"]) #should print -25.2406
print("long: ", city1json["coord"]["lon"]) #should print 130.9889
print("desc: ", city1json["weather"][0]["description"]) # should print 'overcast clouds' (can change when run later)
print("max temp (F): ", city1json["main"]["temp_max"]) #should print 74.43 (can change when run later)
print("humid %: ", city1json["main"]["humidity"]) #should print 46 (can change when run later)
print("wind (mph): ", city1json["wind"]["speed"]) #should print 7.9 (can change when run later)
print("cloudy %: ", city1json["clouds"]["all"]) #should print 96 (can change when run later)
print("country: ", city1json["sys"]["country"]) #should print AU
print("city: ", city1json["name"]) #should print Yulara
print("time: ", datetime.utcfromtimestamp(city1json["dt"]).strftime('%Y-%m-%d %H:%M:%S'))
#everything looks good

{'coord': {'lon': 130.9889, 'lat': -25.2406}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 76.41, 'feels_like': 75.67, 'temp_min': 76.41, 'temp_max': 76.41, 'pressure': 1010, 'humidity': 41, 'sea_level': 1010, 'grnd_level': 953}, 'visibility': 10000, 'wind': {'speed': 9.48, 'deg': 303, 'gust': 14.41}, 'clouds': {'all': 92}, 'dt': 1664858082, 'sys': {'country': 'AU', 'sunrise': 1664830394, 'sunset': 1664874985}, 'timezone': 34200, 'id': 6355222, 'name': 'Yulara', 'cod': 200}
lat:  -25.2406
long:  130.9889
desc:  overcast clouds
max temp (F):  76.41
humid %:  41
wind (mph):  9.48
cloudy %:  92
country:  AU
city:  Yulara
time:  2022-10-04 04:34:42


In [35]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the latitude, longitude, max temp, humidity, cloudiness, wind, country, and weather description
        # Append the city information into the city_data list
        city_data.append({
            "City" : city_weather['name'],
            "Lat" : city_weather['coord']['lat'],
            "Lng" : city_weather['coord']['lon'],
            "Max Temp" : city_weather['main']['temp_max'],
            "Current Description" : city_weather['weather'][0]['description'],
            "Humidity" : city_weather["main"]["humidity"],
            "Cloudiness" : city_weather["clouds"]["all"],
            "Wind Speed" : city_weather["wind"]["speed"],
            "Country" : city_weather["sys"]["country"]
        })
                                     
    # If an error is experienced, skip the city
    except (IndexError, KeyError):
        print(f"City {city} not found. Skipping...")
        pass

# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | yulara,au
Processing Record 2 of Set 1 | hasaki,jp
Processing Record 3 of Set 1 | butaritari,ki
Processing Record 4 of Set 1 | punta arenas,cl
Processing Record 5 of Set 1 | busselton,au
Processing Record 6 of Set 1 | butembo,cd
Processing Record 7 of Set 1 | lorengau,pg
Processing Record 8 of Set 1 | tuy hoa,vn
Processing Record 9 of Set 1 | codrington,ag
City codrington,ag not found. Skipping...
Processing Record 10 of Set 1 | chuy,uy
Processing Record 11 of Set 1 | fort nelson,ca
Processing Record 12 of Set 1 | korla,cn
Processing Record 13 of Set 1 | lagoa,pt
Processing Record 14 of Set 1 | port alfred,za
Processing Record 15 of Set 1 | saint-philippe,re
Processing Record 16 of Set 1 | macusani,pe
Processing Record 17 of Set 1 | rikitea,pf
Processing Record 18 of Set 1 | hithadhoo,mv
Processing Record 19 of Set 1 | qaanaaq,gl
Processing Record 20 of Set 1 | lawigan,ph
Processing Record 21 of 

Processing Record 18 of Set 4 | vaini,to
Processing Record 19 of Set 4 | jaltenango,mx
Processing Record 20 of Set 4 | castro,cl
Processing Record 21 of Set 4 | nyakabindi,tz
Processing Record 22 of Set 4 | port hardy,ca
Processing Record 23 of Set 4 | tautira,pf
Processing Record 24 of Set 4 | wajir,ke
Processing Record 25 of Set 4 | saskylakh,ru
Processing Record 26 of Set 4 | bambous virieux,mu
Processing Record 27 of Set 4 | san vicente de canete,pe
Processing Record 28 of Set 4 | progreso,mx
Processing Record 29 of Set 4 | sinnamary,gf
Processing Record 30 of Set 4 | killybegs,ie
Processing Record 31 of Set 4 | olecko,pl
Processing Record 32 of Set 4 | coolum beach,au
Processing Record 33 of Set 4 | tsihombe,mg
City tsihombe,mg not found. Skipping...
Processing Record 34 of Set 4 | pangnirtung,ca
Processing Record 35 of Set 4 | ilebo,cd
Processing Record 36 of Set 4 | porosozero,ru
Processing Record 37 of Set 4 | goedereede,nl
Processing Record 38 of Set 4 | la ronge,ca
Processing

Processing Record 37 of Set 7 | kyra,ru
Processing Record 38 of Set 7 | rosarito,mx
Processing Record 39 of Set 7 | tabarqah,tn
City tabarqah,tn not found. Skipping...
Processing Record 40 of Set 7 | papalotla,mx
Processing Record 41 of Set 7 | fountain hills,us
Processing Record 42 of Set 7 | huntington,us
Processing Record 43 of Set 7 | soe,id
Processing Record 44 of Set 7 | petropavlovsk-kamchatskiy,ru
Processing Record 45 of Set 7 | yaroslavskiy,ru
Processing Record 46 of Set 7 | zhigalovo,ru
Processing Record 47 of Set 7 | atar,mr
Processing Record 48 of Set 7 | east london,za
Processing Record 49 of Set 7 | yomitan,jp
City yomitan,jp not found. Skipping...
Processing Record 50 of Set 7 | cherskiy,ru
Processing Record 1 of Set 8 | el wasta,eg
City el wasta,eg not found. Skipping...
Processing Record 2 of Set 8 | bedele,et
Processing Record 3 of Set 8 | berbera,so
Processing Record 4 of Set 8 | port-gentil,ga
Processing Record 5 of Set 8 | beyla,gn
Processing Record 6 of Set 8 | ve

Processing Record 5 of Set 11 | burica,pa
City burica,pa not found. Skipping...
Processing Record 6 of Set 11 | manavalakurichi,in
Processing Record 7 of Set 11 | chicama,pe
Processing Record 8 of Set 11 | espanola,us
Processing Record 9 of Set 11 | barawe,so
City barawe,so not found. Skipping...
Processing Record 10 of Set 11 | gumdag,tm
Processing Record 11 of Set 11 | hualmay,pe
Processing Record 12 of Set 11 | sassandra,ci
Processing Record 13 of Set 11 | pareora,nz
Processing Record 14 of Set 11 | dickinson,us
Processing Record 15 of Set 11 | shimanovsk,ru
Processing Record 16 of Set 11 | luganville,vu
Processing Record 17 of Set 11 | coihaique,cl
Processing Record 18 of Set 11 | mikuni,jp
Processing Record 19 of Set 11 | yunjinghong,cn
City yunjinghong,cn not found. Skipping...
Processing Record 20 of Set 11 | georgetown,sh
Processing Record 21 of Set 11 | severo-yeniseyskiy,ru
Processing Record 22 of Set 11 | longjiang,cn
Processing Record 23 of Set 11 | balakirevo,ru
Processing

City talawdi,sd not found. Skipping...
Processing Record 25 of Set 14 | la macarena,co
Processing Record 26 of Set 14 | benxi,cn
Processing Record 27 of Set 14 | giyani,za
Processing Record 28 of Set 14 | stornoway,gb
Processing Record 29 of Set 14 | hovd,mn
Processing Record 30 of Set 14 | barra do corda,br
Processing Record 31 of Set 14 | launceston,au
Processing Record 32 of Set 14 | korsakovo,ru
Processing Record 33 of Set 14 | uren,ru
Processing Record 34 of Set 14 | huntingdon,gb
Processing Record 35 of Set 14 | malmesbury,za
Processing Record 36 of Set 14 | castelo do piaui,br
Processing Record 37 of Set 14 | sombrio,br
Processing Record 38 of Set 14 | lokosovo,ru
Processing Record 39 of Set 14 | puerto colombia,co
Processing Record 40 of Set 14 | pueblo nuevo,mx
Processing Record 41 of Set 14 | pa sang,th
Processing Record 42 of Set 14 | mount gambier,au
Processing Record 43 of Set 14 | banda aceh,id
Processing Record 44 of Set 14 | sola,vu
Processing Record 45 of Set 14 | haik

In [36]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)

682

## Add the weather data to a new DataFrame.

In [37]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [38]:
# Display sample data
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,Yulara,-25.2406,130.9889,76.41,overcast clouds,41,92,9.48,AU
1,Hasaki,35.7333,140.8333,86.29,broken clouds,68,55,24.36,JP
2,Butaritari,3.0707,172.7902,82.09,scattered clouds,71,49,14.29,KI
3,Punta Arenas,-53.1500,-70.9167,42.91,broken clouds,81,75,6.91,CL
4,Busselton,-33.6500,115.3333,63.01,clear sky,60,0,13.65,AU
5,Butembo,0.1500,29.2833,58.96,overcast clouds,92,100,1.81,CD
6,Lorengau,-2.0226,147.2712,86.05,broken clouds,68,63,11.32,PG
7,Tuy Hoa,13.0833,109.3000,78.73,overcast clouds,90,100,7.00,VN
8,Chui,-33.6971,-53.4616,53.96,overcast clouds,64,90,7.76,UY
9,Fort Nelson,58.8053,-122.7002,40.59,few clouds,75,20,0.00,CA


In [39]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Current Description', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Country'],
      dtype='object')

In [41]:
# Reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
city_data_df = city_data_df[["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]]

# Display sample data
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Yulara,AU,-25.2406,130.9889,76.41,41,92,9.48,overcast clouds
1,Hasaki,JP,35.7333,140.8333,86.29,68,55,24.36,broken clouds
2,Butaritari,KI,3.0707,172.7902,82.09,71,49,14.29,scattered clouds
3,Punta Arenas,CL,-53.1500,-70.9167,42.91,81,75,6.91,broken clouds
4,Busselton,AU,-33.6500,115.3333,63.01,60,0,13.65,clear sky
5,Butembo,CD,0.1500,29.2833,58.96,92,100,1.81,overcast clouds
6,Lorengau,PG,-2.0226,147.2712,86.05,68,63,11.32,broken clouds
7,Tuy Hoa,VN,13.0833,109.3000,78.73,90,100,7.00,overcast clouds
8,Chui,UY,-33.6971,-53.4616,53.96,64,90,7.76,overcast clouds
9,Fort Nelson,CA,58.8053,-122.7002,40.59,75,20,0.00,few clouds


In [42]:
# Display the data types of each column by using the dtypes Pandas function
city_data_df.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [46]:
# Set the output file name
outfile = "WeatherPy_Database.csv"

# Export the city_data DataFrame into a CSV file
city_data_df.to_csv(outfile, index_label="City_ID")